## **Card Definition Extractor**

Standalone version with directions: https://sucker.severian.dev

I've gotten into making models at [trashpanda-org](https://huggingface.co/trashpanda-org), check out hasnonname's [Mullein](https://huggingface.co/trashpanda-org/MS-24B-Mullein-v0)!

> _lmk on Discord if you have any issues while using this - Severian_

---

**Changelog:**
- v0.2: fixed to handle Janitor making changes due to R1 handling.


In [ ]:
# @title Card Definition Extractor

# @markdown Directions for use:
# @markdown - If enabled, starts the proxy in character card extraction mode.
# @markdown - Use the proxy as normal, and start a new chat with your character of choice.
# @markdown - After sending the first message, the proxy will process the character card in v1 format
# @markdown - Stop the proxy and Colab will download the JSON file on your device
# @markdown - Your custom prompt will appear on the description field so this is best used with a cleared-out custom prompt section on janitor.ai
# @markdown - You can start multiple new chats and send messages for the extractor to capture cards, and when you stop the notebook, it will download all extracted files at once.

# @markdown **Select Tunnel Provider**
tunnel_provider = "Cloudflare"  # @param ["Cloudflare", "Localtunnel", "Ngrok"]

# @markdown **Ngrok Auth Token**: If using ngrok, sign up for an auth token at https://dashboard.ngrok.com/signup
ngrok_auth_token = ""  # @param {type:"string"}

card_definition_extractor = True
!pip install flask-cors

import json
import requests
import time
from flask import Flask, request, jsonify
from flask_cors import CORS
import re
import tempfile
import os

app = Flask(__name__)
CORS(app)

# Depending on the provider, set up the tunnel
if tunnel_provider == "Cloudflare":
    !pip install flask-cors flask_cloudflared
    from flask_cloudflared import run_with_cloudflared
    run_with_cloudflared(app)
elif tunnel_provider == "Localtunnel":
    !pip install flask-cors flask_localtunnel
    from flask_lt import run_with_lt
    run_with_lt(app)
elif tunnel_provider == "Ngrok":
    !pip install flask-cors pyngrok==7.1.2
    from pyngrok import ngrok
    if ngrok_auth_token.strip():
        ngrok.set_auth_token(ngrok_auth_token.strip())
    public_url = ngrok.connect(5000).public_url
    print("Public URL:", public_url)

def extract_between_tags(content, tag):
    """
    Extracts content between XML-like tags.
    Returns empty string if tag not found.
    """
    start_tag = f"<{tag}>"
    end_tag = f"</{tag}>"
    start_idx = content.find(start_tag)
    if start_idx == -1:
        return ""
    
    end_idx = content.find(end_tag, start_idx)
    if end_idx == -1:
        return ""
    
    return content[start_idx + len(start_tag):end_idx].strip()

def find_tags_between(content, start_marker, end_marker):
    """
    Finds all XML-like tags and their content between two marker tags.
    Returns list of {tag, content} dictionaries.
    """
    start_idx = content.find(f"<{start_marker}>")
    if start_idx == -1:
        return []
    
    end_idx = content.find(f"<{end_marker}>")
    if end_idx == -1:
        return []
    
    section = content[start_idx + len(start_marker) + 2:end_idx]
    tag_regex = r"<([^/>]+)>([^<]+)</\1>"
    matches = re.finditer(tag_regex, section)
    
    return [{"tag": match.group(1), "content": match.group(2).strip()} for match in matches]

def extract_card_data(messages):
    content0 = messages[0]["content"]
    content1 = messages[2]["content"]

    # Find all persona tags between system and scenario, take the last one as character
    personas = find_tags_between(content0, "system", "scenario")
    char_persona = personas[-1] if personas else {"tag": "", "content": ""}
    char_name = char_persona["tag"]

    card_data = {
        "name": char_name,
        "description": char_persona["content"],
        "scenario": extract_between_tags(content0, "scenario"),
        "mes_example": extract_between_tags(content0, "example_dialogs"),
        "personality": "",  # This field isn't used in the new format
        "first_mes": content1
    }

    # Replace character name with placeholder in all fields
    def safe_replace(text, old, new):
        return text.replace(old, new) if old else text

    for field in card_data:
        if field != "name":  # Exclude the "name" field
            val = card_data[field]
            val = safe_replace(val, char_name, "{{char}}")
            card_data[field] = val

    return card_data

@app.route('/', methods=['GET'])
def default():
    return {"status": "online"}

@app.route('/', methods=['POST'])
def process_card():
    body = request.json
    if 'messages' not in body:
        return jsonify(error="Missing 'messages' in request body"), 400

    if card_definition_extractor and len(body["messages"]) >= 2:
        card_data = extract_card_data(body["messages"])
        # If running in Colab, download the file
        try:
            from google.colab import files
            import tempfile
            temp_json = tempfile.NamedTemporaryFile(delete=False, suffix=".json")
            with open(temp_json.name, 'w', encoding='utf-8') as f:
                json.dump(card_data, f, ensure_ascii=False, indent=2)
            print("Card definition JSON created at:", temp_json.name)
            files.download(temp_json.name)
        except ImportError:
            pass  # Not in Colab, just return JSON

        return jsonify(card_data), 200
    else:
        return jsonify(status="Card definition extractor not enabled or insufficient messages"), 200

if __name__ == '__main__':
    if tunnel_provider != "Cloudflare":
        print('\n Colab IP: ', end='')
        !curl ipecho.net/plain
        print('\n')
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on https://little-disputes-posting-palmer.trycloudflare.com
 * Traffic stats available on http://127.0.0.1:8396/metrics


INFO:werkzeug:127.0.0.1 - - [04/Feb/2025 22:53:13] "OPTIONS / HTTP/1.1" 200 -


Card definition JSON created at: /tmp/tmpynlda8kv.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

INFO:werkzeug:127.0.0.1 - - [04/Feb/2025 22:53:14] "POST / HTTP/1.1" 200 -
